# Hyperparameter Optimization All models

## Rationale:
In order to make a fair comparisson in performance, we are to tune the hyperparamters for all the models so that we can optimize the performance of each model and then compared then to see if we may be affecting because a bad initial setup.

## Methodology:
We conducted hyperparameter optimization using the Optuna library, which provides a powerful and efficient platform for automated hyperparameter tuning.

We are to Use the ```LightGBM Integration``` which optimizes the hyperparamters in a sequential way and focuses on the parameters that have the highest impact on model performance for Boosting models:



* **lambda_l1**
* **lambda_l2**
* **num_leaves**
* **feature_fraction**
* **bagging_fraction**
* **bagging_freq**
* **min_child_samples**

For the **num_estimators** and **learning_rate** we are to use the ones found on the best model optimization.

## Conclusions:
The results of our hyperparameter optimization study yielded valuable insights into improving the LightGBM model's performance. For all the models we achived an increased in performance as compared with the initial model comparisson. (see ```feature-selection``` notebook).


| Model               | out_of_fold | validation |
|---------------------|------------|------------|
| fw [32]             | 0.863464   | 0.867463   |
| boruta [46]         | 0.864788   | 0.867128   |
| all features [115] | 0.862991   | 0.867067   |
| ensemble [55]       | 0.862760   | 0.865526   |
| base_features [10]  | 0.864081   | 0.865510   |





In [16]:
%load_ext autoreload
%autoreload 2

import cloudpickle as cp

import numpy as np
import pandas as pd

import lightgbm  as lgbm
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split
from copy import deepcopy

from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import roc_auc_score

import warnings;warnings.filterwarnings("ignore")

from optuna import visualization as optunaviz

import optuna
import optuna.integration.lightgbm as lgb

from lightgbm import early_stopping
from lightgbm import log_evaluation
from optuna.logging import set_verbosity

set_verbosity(optuna.logging.ERROR)

import sys
sys.path.append("../")

# local imports
from src.learner_params import target_column, boruta_learner_params, test_params
from utils.functions__utils import find_constraint

from utils.feature_selection_lists import fw_features, boruta_features, ensemble_features
from utils.features_lists import all_features_list, base_features

from src.learner_params import MODEL_PARAMS

from utils.functions__training import model_pipeline

from src.learner_params import params_all, params_ensemble, params_fw, params_original

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
train_df = pd.read_pickle("../data/train_df.pkl")
validation_df = pd.read_pickle("../data/validation_df.pkl")

train, test = train_test_split(train_df, random_state=42, test_size=.2)

### All features

In [5]:
%%time

dtrain = lgbm.Dataset(train[all_features_list], label=train[target_column])
dval = lgb.Dataset(test[all_features_list], label=test[target_column])

params = {
    "objective": "binary",
    "metric": "auc",
    "verbosity": -1,
    "boosting_type": "gbdt",
    "n_estimators":5926,
    "learning_rate": 0.005603627873630697
}

bst = lgb.train(
    params,
    dtrain,
    valid_sets=[dtrain, dval],
    callbacks=[early_stopping(150)],
    show_progress_bar = True
)

preds = bst.predict(test[all_features_list],num_iteration=bst.best_iteration)
score = roc_auc_score(test[target_column], preds)

all_features_best_params = bst.params
print("Best params:", all_features_best_params)



feature_fraction, val_score: -inf:   0%|                  | 0/7 [00:00<?, ?it/s]

Training until validation scores don't improve for 150 rounds


feature_fraction, val_score: 0.871090:  14%|8     | 1/7 [00:12<01:15, 12.56s/it]

Early stopping, best iteration is:
[832]	valid_0's auc: 0.891391	valid_1's auc: 0.87109
Training until validation scores don't improve for 150 rounds


feature_fraction, val_score: 0.871358:  29%|#7    | 2/7 [00:24<01:01, 12.25s/it]

...

min_data_in_leaf, val_score: 0.871743:  80%|####8 | 4/5 [00:55<00:13, 13.89s/it]

Early stopping, best iteration is:
[937]	valid_0's auc: 0.892586	valid_1's auc: 0.871542
Training until validation scores don't improve for 150 rounds


min_data_in_leaf, val_score: 0.871743: 100%|######| 5/5 [01:08<00:00, 13.61s/it]

Early stopping, best iteration is:
[779]	valid_0's auc: 0.885093	valid_1's auc: 0.871508


Best params: {'objective': 'binary', 'metric': 'auc', 'verbosity': -1, 'boosting_type': 'gbdt', 'learning_rate': 0.005603627873630697, 'feature_pre_filter': False, 'lambda_l1': 0.022801469789147367, 'lambda_l2': 0.004898476816241659, 'num_leaves': 31, 'feature_fraction': 0.4, 'bagging_fraction': 0.8079749072343313, 'bagging_freq': 5, 'min_child_samples': 20, 'num_iterations': 5926}
CPU times: user 40min 39s, sys: 5min 53s, total: 46min 33s
Wall time: 15min 14s


In [6]:
all_features_best_params

{'objective': 'binary',
 'metric': 'auc',
 'verbosity': -1,
 'boosting_type': 'gbdt',
 'learning_rate': 0.005603627873630697,
 'feature_pre_filter': False,
 'lambda_l1': 0.022801469789147367,
 'lambda_l2': 0.004898476816241659,
 'num_leaves': 31,
 'feature_fraction': 0.4,
 'bagging_fraction': 0.8079749072343313,
 'bagging_freq': 5,
 'min_child_samples': 20,
 'num_iterations': 5926}

### MrMr Features

In [7]:
%%time
dtrain = lgbm.Dataset(train[fw_features], label=train[target_column])
dval = lgb.Dataset(test[fw_features], label=test[target_column])
params = {
    "objective": "binary",
    "metric": "auc",
    "verbosity": -1,
    "boosting_type": "gbdt",
    "n_estimators":5926,
    "learning_rate": 0.005603627873630697
}

bst = lgb.train(
    params,
    dtrain,
    valid_sets=[dtrain, dval],
    callbacks=[early_stopping(150)],
    show_progress_bar = True
)

preds = bst.predict(test[fw_features],num_iteration=bst.best_iteration)
score = roc_auc_score(test[target_column], preds)

fw_features_best_params = bst.params
print("Best params:", fw_features_best_params)



feature_fraction, val_score: -inf:   0%|                  | 0/7 [00:00<?, ?it/s]

Training until validation scores don't improve for 150 rounds


feature_fraction, val_score: 0.870538:  14%|8     | 1/7 [00:10<01:00, 10.16s/it]

Early stopping, best iteration is:
[848]	valid_0's auc: 0.891962	valid_1's auc: 0.870538
Training until validation scores don't improve for 150 rounds


feature_fraction, val_score: 0.870630:  29%|#7    | 2/7 [00:19<00:49,  9.86s/it]

...

min_data_in_leaf, val_score: 0.872275:  80%|####8 | 4/5 [01:00<00:14, 14.70s/it]

Early stopping, best iteration is:
[1093]	valid_0's auc: 0.879975	valid_1's auc: 0.872061
Training until validation scores don't improve for 150 rounds


min_data_in_leaf, val_score: 0.872275: 100%|######| 5/5 [01:17<00:00, 15.48s/it]

Early stopping, best iteration is:
[1731]	valid_0's auc: 0.886282	valid_1's auc: 0.872206


Best params: {'objective': 'binary', 'metric': 'auc', 'verbosity': -1, 'boosting_type': 'gbdt', 'learning_rate': 0.005603627873630697, 'feature_pre_filter': False, 'lambda_l1': 4.1326356980799586e-05, 'lambda_l2': 0.6016241539010313, 'num_leaves': 18, 'feature_fraction': 0.4, 'bagging_fraction': 0.9403276152498784, 'bagging_freq': 4, 'min_child_samples': 50, 'num_iterations': 5926}
CPU times: user 38min 48s, sys: 5min 38s, total: 44min 26s
Wall time: 15min 5s


In [8]:
fw_features_best_params

{'objective': 'binary',
 'metric': 'auc',
 'verbosity': -1,
 'boosting_type': 'gbdt',
 'learning_rate': 0.005603627873630697,
 'feature_pre_filter': False,
 'lambda_l1': 4.1326356980799586e-05,
 'lambda_l2': 0.6016241539010313,
 'num_leaves': 18,
 'feature_fraction': 0.4,
 'bagging_fraction': 0.9403276152498784,
 'bagging_freq': 4,
 'min_child_samples': 50,
 'num_iterations': 5926}

### Ensemble features

In [9]:
%%time
dtrain = lgbm.Dataset(train[ensemble_features], label=train[target_column])
dval = lgb.Dataset(test[ensemble_features], label=test[target_column])

params = {
    "objective": "binary",
    "metric": "auc",
    "verbosity": -1,
    "boosting_type": "gbdt",
    "n_estimators":5926,
    "learning_rate": 0.005603627873630697
}

bst = lgb.train(
    params,
    dtrain,
    valid_sets=[dtrain, dval],
    callbacks=[early_stopping(150)],
    show_progress_bar = True
)

preds = bst.predict(test[ensemble_features],num_iteration=bst.best_iteration)
score = roc_auc_score(test[target_column], preds)

ensemble_features_best_params = bst.params
print("Best params:", ensemble_features_best_params)



feature_fraction, val_score: -inf:   0%|                  | 0/7 [00:00<?, ?it/s]

Training until validation scores don't improve for 150 rounds


feature_fraction, val_score: 0.870887:  14%|8     | 1/7 [00:11<01:11, 11.96s/it]

Early stopping, best iteration is:
[915]	valid_0's auc: 0.894268	valid_1's auc: 0.870887
Training until validation scores don't improve for 150 rounds


feature_fraction, val_score: 0.871175:  29%|#7    | 2/7 [00:22<00:56, 11.36s/it]

...

min_data_in_leaf, val_score: 0.871586:  80%|####8 | 4/5 [01:31<00:23, 23.35s/it]

Early stopping, best iteration is:
[3520]	valid_0's auc: 0.87406	valid_1's auc: 0.8715
Training until validation scores don't improve for 150 rounds


min_data_in_leaf, val_score: 0.871586: 100%|######| 5/5 [01:56<00:00, 23.34s/it]

Early stopping, best iteration is:
[3526]	valid_0's auc: 0.874169	valid_1's auc: 0.871439


Best params: {'objective': 'binary', 'metric': 'auc', 'verbosity': -1, 'boosting_type': 'gbdt', 'learning_rate': 0.005603627873630697, 'feature_pre_filter': False, 'lambda_l1': 0.0006278595123135671, 'lambda_l2': 8.715856738389705, 'num_leaves': 6, 'feature_fraction': 0.5, 'bagging_fraction': 1.0, 'bagging_freq': 0, 'min_child_samples': 20, 'num_iterations': 5926}
CPU times: user 1h 1min 16s, sys: 7min 24s, total: 1h 8min 40s
Wall time: 20min 18s


In [14]:
ensemble_features_best_params

{'objective': 'binary',
 'metric': 'auc',
 'verbosity': -1,
 'boosting_type': 'gbdt',
 'learning_rate': 0.005603627873630697,
 'feature_pre_filter': False,
 'lambda_l1': 0.0006278595123135671,
 'lambda_l2': 8.715856738389705,
 'num_leaves': 6,
 'feature_fraction': 0.5,
 'bagging_fraction': 1.0,
 'bagging_freq': 0,
 'min_child_samples': 20,
 'num_iterations': 5926}

### Base model features

In [12]:
%%time
dtrain = lgbm.Dataset(train[base_features], label=train[target_column])
dval = lgb.Dataset(test[base_features], label=test[target_column])

params = {
    "objective": "binary",
    "metric": "auc",
    "verbosity": -1,
    "boosting_type": "gbdt",
    "n_estimators":5926,
    "learning_rate": 0.005603627873630697
}

bst = lgb.train(
    params,
    dtrain,
    valid_sets=[dtrain, dval],
    callbacks=[early_stopping(150)],
    show_progress_bar = True
)

preds = bst.predict(test[base_features],num_iteration=bst.best_iteration)
score = roc_auc_score(test[target_column], preds)

base_features_best_params = bst.params
print("Best params:", base_features_best_params)



feature_fraction, val_score: -inf:   0%|                  | 0/7 [00:00<?, ?it/s]

Training until validation scores don't improve for 150 rounds


feature_fraction, val_score: 0.871848:  14%|8     | 1/7 [00:07<00:46,  7.69s/it]

Early stopping, best iteration is:
[734]	valid_0's auc: 0.882724	valid_1's auc: 0.871848
Training until validation scores don't improve for 150 rounds


feature_fraction, val_score: 0.871848:  29%|#7    | 2/7 [00:18<00:48,  9.75s/it]

...

min_data_in_leaf, val_score: 0.872656:  80%|####8 | 4/5 [00:38<00:09,  9.44s/it]

Early stopping, best iteration is:
[1028]	valid_0's auc: 0.877129	valid_1's auc: 0.872474
Training until validation scores don't improve for 150 rounds


min_data_in_leaf, val_score: 0.872656: 100%|######| 5/5 [00:48<00:00,  9.68s/it]

Early stopping, best iteration is:
[1117]	valid_0's auc: 0.877538	valid_1's auc: 0.8726


Best params: {'objective': 'binary', 'metric': 'auc', 'verbosity': -1, 'boosting_type': 'gbdt', 'learning_rate': 0.005603627873630697, 'feature_pre_filter': False, 'lambda_l1': 2.1179144447747353e-08, 'lambda_l2': 3.393255954585769e-08, 'num_leaves': 18, 'feature_fraction': 0.58, 'bagging_fraction': 0.9991117695058381, 'bagging_freq': 1, 'min_child_samples': 20, 'num_iterations': 5926}
CPU times: user 27min 44s, sys: 3min 47s, total: 31min 31s
Wall time: 10min 44s


In [13]:
base_features_best_params

{'objective': 'binary',
 'metric': 'auc',
 'verbosity': -1,
 'boosting_type': 'gbdt',
 'learning_rate': 0.005603627873630697,
 'feature_pre_filter': False,
 'lambda_l1': 2.1179144447747353e-08,
 'lambda_l2': 3.393255954585769e-08,
 'num_leaves': 18,
 'feature_fraction': 0.58,
 'bagging_fraction': 0.9991117695058381,
 'bagging_freq': 1,
 'min_child_samples': 20,
 'num_iterations': 5926}

### Model training

In [17]:
fw_logs = model_pipeline(train_df = train_df,
                            validation_df = validation_df,
                            params = params_fw,
                            target_column = target_column,
                            features = fw_features,
                            cv = 3,
                            random_state = 42,
                            apply_shap = False,
                            save_estimator_path=None
                          )

2023-10-08T18:24:30 | INFO | Starting pipeline: Generating 3 k-fold training...
2023-10-08T18:24:30 | INFO | Training for fold 1
2023-10-08T18:24:58 | INFO | Training for fold 2
2023-10-08T18:25:26 | INFO | Training for fold 3
2023-10-08T18:25:55 | INFO | CV training finished!
2023-10-08T18:25:55 | INFO | Training the model in the full dataset...
2023-10-08T18:26:28 | INFO | Training process finished!
2023-10-08T18:26:28 | INFO | Calculating metrics...
2023-10-08T18:26:28 | INFO | Full process finished in 1.96 minutes.
2023-10-08T18:26:28 | INFO | Saving the predict function.
2023-10-08T18:26:28 | INFO | Predict function saved.


In [18]:
ensemble_logs = model_pipeline(train_df = train_df,
                            validation_df = validation_df,
                            params = test_params,
                            target_column = target_column,
                            features = ensemble_features,
                            cv = 3,
                            random_state = 42,
                            apply_shap = False,
                            save_estimator_path=None
                          )

2023-10-08T18:26:28 | INFO | Starting pipeline: Generating 3 k-fold training...
2023-10-08T18:26:28 | INFO | Training for fold 1
2023-10-08T18:26:46 | INFO | Training for fold 2
2023-10-08T18:27:04 | INFO | Training for fold 3
2023-10-08T18:27:22 | INFO | CV training finished!
2023-10-08T18:27:22 | INFO | Training the model in the full dataset...
2023-10-08T18:27:42 | INFO | Training process finished!
2023-10-08T18:27:42 | INFO | Calculating metrics...
2023-10-08T18:27:42 | INFO | Full process finished in 1.24 minutes.
2023-10-08T18:27:42 | INFO | Saving the predict function.
2023-10-08T18:27:42 | INFO | Predict function saved.


In [19]:
all_logs = model_pipeline(train_df = train_df,
                            validation_df = validation_df,
                            params = params_all,
                            target_column = target_column,
                            features = all_features_list,
                            cv = 3,
                            random_state = 42,
                            apply_shap = False,
                            save_estimator_path=None
                          )

2023-10-08T18:27:43 | INFO | Starting pipeline: Generating 3 k-fold training...
2023-10-08T18:27:43 | INFO | Training for fold 1
2023-10-08T18:28:21 | INFO | Training for fold 2
2023-10-08T18:29:01 | INFO | Training for fold 3
2023-10-08T18:29:41 | INFO | CV training finished!
2023-10-08T18:29:41 | INFO | Training the model in the full dataset...
2023-10-08T18:30:30 | INFO | Training process finished!
2023-10-08T18:30:30 | INFO | Calculating metrics...
2023-10-08T18:30:30 | INFO | Full process finished in 2.80 minutes.
2023-10-08T18:30:30 | INFO | Saving the predict function.
2023-10-08T18:30:30 | INFO | Predict function saved.


In [20]:
boruta_logs = model_pipeline(train_df = train_df,
                            validation_df = validation_df,
                            params = MODEL_PARAMS,
                            target_column = target_column,
                            features = boruta_features,
                            cv = 3,
                            random_state = 42,
                            apply_shap = False,
                            save_estimator_path=None
                          )

2023-10-08T18:30:30 | INFO | Starting pipeline: Generating 3 k-fold training...
2023-10-08T18:30:30 | INFO | Training for fold 1
2023-10-08T18:32:08 | INFO | Training for fold 2
2023-10-08T18:33:47 | INFO | Training for fold 3
2023-10-08T18:35:26 | INFO | CV training finished!
2023-10-08T18:35:26 | INFO | Training the model in the full dataset...
2023-10-08T18:37:47 | INFO | Training process finished!
2023-10-08T18:37:47 | INFO | Calculating metrics...
2023-10-08T18:37:47 | INFO | Full process finished in 7.28 minutes.
2023-10-08T18:37:47 | INFO | Saving the predict function.
2023-10-08T18:37:47 | INFO | Predict function saved.


In [22]:
base_logs = model_pipeline(train_df = train_df,
                            validation_df = validation_df,
                            params = params_original,
                            target_column = target_column,
                            features = base_features,
                            cv = 3,
                            random_state = 42,
                            apply_shap = False,
                            save_estimator_path=None
                          )

2023-10-08T18:41:34 | INFO | Starting pipeline: Generating 3 k-fold training...
2023-10-08T18:41:34 | INFO | Training for fold 1
2023-10-08T18:41:56 | INFO | Training for fold 2
2023-10-08T18:42:17 | INFO | Training for fold 3
2023-10-08T18:42:38 | INFO | CV training finished!
2023-10-08T18:42:38 | INFO | Training the model in the full dataset...
2023-10-08T18:43:04 | INFO | Training process finished!
2023-10-08T18:43:04 | INFO | Calculating metrics...
2023-10-08T18:43:04 | INFO | Full process finished in 1.49 minutes.
2023-10-08T18:43:04 | INFO | Saving the predict function.
2023-10-08T18:43:04 | INFO | Predict function saved.


### Model evaluation

In [23]:
model_metrics  ={}
models = [base_logs, all_logs, fw_logs, boruta_logs, ensemble_logs]
names = ["base_features", "all features", "fw", "boruta", "ensemble"]
sizes = [len(base_features),len(all_features_list), len(fw_features), len(boruta_features), len(ensemble_features)]

for model, name, size in zip(models, names, sizes):
    model_metrics[f"{name} [{size}]"] = model["metrics"]["roc_auc"]
pd.DataFrame(model_metrics).T.sort_values(by = "validation", ascending = False)

,out_of_fold,validation
fw [32],0.863464,0.867463
boruta [46],0.864788,0.867128
all features [115],0.862991,0.867067
ensemble [55],0.862760,0.865526
base_features [10],0.864081,0.865510
